In [1]:
import datetime as dt
import polars as pl

from factor_momentum import RollingPCA, FactorMomentumSignal, FACTORS
from sf_quant.data import load_factors


/home/boobus/projects/factor_momentum/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-10 09:30:16,067	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
start = dt.date(2001,1,1); end = dt.date(2002,1,1); factor_returns = load_factors(start,end, FACTORS).lazy()
pca_engine = RollingPCA(n_components=5, window=30)

In [4]:
pca_engine._fit_to_dates(start,end,start,factor_returns)

In [ ]:
pca_engine._transform_date(dt.date(2001,12,27),factor_returns)

In [5]:
r = (
    factor_returns
    .filter(pl.col('date').eq(dt.date(2001,12,27)))
    .drop('date')
    .collect()       
)
r


USSLOWL_BETA,USSLOWL_DIVYILD,USSLOWL_EARNQLTY,USSLOWL_EARNYILD,USSLOWL_GROWTH,USSLOWL_LEVERAGE,USSLOWL_LIQUIDTY,USSLOWL_MGMTQLTY,USSLOWL_MIDCAP,USSLOWL_PROFIT,USSLOWL_PROSPECT,USSLOWL_RESVOL,USSLOWL_SIZE,USSLOWL_VALUE
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.224071,0.0141412,-0.131906,0.122072,0.02166,0.065035,0.021172,0.087335,-0.074526,0.027233,0.0855313,0.2568816,-0.045176,-0.065525


In [9]:
r_t = pca_engine.pca_model.transform(r.to_numpy())
r_t

array([[ 0.16661155,  0.07249787, -0.12412394,  0.00727921,  0.12932517]])

In [ ]:
a = pl.DataFrame(r_t).with_columns(pl.lit(dt.date()))
a

column_0,column_1,column_2,column_3,column_4
f64,f64,f64,f64,f64
0.166612,0.072498,-0.124124,0.007279,0.129325


In [5]:
L = pca_engine.pca_model.components_

In [6]:
pc_weights = [1,0,1,-1,-1]

In [7]:
factor_weights = pc_weights @ L

In [8]:
print(factor_weights)

[-0.19003958  0.46651775  0.60943243  0.40163084 -0.34132498  0.07378349
 -0.65025062 -1.12053034  0.05664338  0.01048483 -0.26934533  0.09090733
 -0.76485347 -0.86257741]
